In [93]:
import random

from DrawSolution import draw_pallet
from class_item import Item
import copy
import time
import random
from matplotlib import pyplot as plt
from matplotlib import patches


eps = 1

poligon1 = [[0,1,0], [1,1,1], [0,1,0]]
poligon2 = [[1,1], [1,0]]
poligon3 = [[1,1,1], [0,1,0]]

points_1 = [[0,3], [2,0], [3,1], [2,1], [1, 4]]
points_2 = [[0,0], [1,0], [1,8], [0,8]]
points_3 = [[0,3], [2,0], [2,1], [3,2], [1, 4]]

poligon_matrix = [[1,1,1,1], [1,1,1,1], [1,1,1,1]]
short_poligon_matrix = [[1,1,1,1,1,1,1,1]]

poligons = [copy.deepcopy(points_2), copy.deepcopy(points_2), copy.deepcopy(points_1), copy.deepcopy(points_1), copy.deepcopy(points_1), copy.deepcopy(points_1), copy.deepcopy(points_1), copy.deepcopy(points_1), copy.deepcopy(points_1), copy.deepcopy(points_1)]

# poligons = [copy.deepcopy(points_2),copy.deepcopy(points_2)]
items = [Item(i, poligons[i]) for i in range(len(poligons))]



items[0].matrix = short_poligon_matrix
items[1].matrix = short_poligon_matrix




for item in items:
    if item.matrix is None:
        item.matrix = poligon_matrix

matrix = [[0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0]]


In [94]:


def find_lb_coordinates(items, eps):
    for item in items:
        item.lb_x = item.lb_x * eps
        item.lb_y = item.lb_y * eps
    return items

def fit_item(pallet, item):
    for i in range(len(pallet[0])):
        for j in range(len(pallet)):
            exit = False
            # условия, что объект влезает
            if len(item.matrix) + i <= len(pallet) and len(item.matrix[0]) + j <= len(pallet[0]):
                # располагаем объект
                for p in range(len(item.matrix)):
                    for k in range(len(item.matrix[0])):
                        # решаем есть ли пересечение
                        if pallet[i+p][j+k] + item.matrix[p][k] == 2:
                            exit = True
                            break
                    if exit:
                        break
            else:
                exit = True
            # если пересечений нет и элемент влезает, то добавляем его
            if not exit:
                item.lb_x = i
                item.lb_y = j
                for p in range(len(item.matrix)):
                    for k in range(len(item.matrix[0])):
                        pallet[i+p][j+k] += item.matrix[p][k]
                break
        if not exit:
            break

    return pallet, exit

def fit_pallets(matrix, items, eps):
    pallets = []
    pallets.append(copy.deepcopy(matrix))
    for item in items:
        i=0
        exit = True
        while exit and i<len(pallets):
            pallets[i], exit = fit_item(pallets[i], item)
            if exit and i==(len(pallets)-1):
                pallets.append(copy.deepcopy(matrix))
            if not exit:
                item.pallet_number = i
                # print(i)
            i+=1
    find_lb_coordinates(items, eps)

    return pallets

In [95]:
def swap(list, pos1, pos2):

    list[pos1], list[pos2] = list[pos2], list[pos1]
    return list

def locSearch(matrix, poligons, eps):
    # print(eps)

    # print(poligons[0].id)
    # print(swap(poligons, 0 ,1)[0].id)
    n = len( poligons)

    objVal = len(fit_pallets(matrix,  poligons, eps))

    poligonsBest = copy.deepcopy(poligons)

    betterNeighboor = (0,0)
    stop = False
    while not stop:

        stop = True
        for i in range(n):
            for j in range(i + 1, n):
                # надо сделать, что бы два раза не проходила
                pal = fit_pallets(matrix, swap(copy.deepcopy(poligonsBest), i, j), eps)
                # print(items[0].pallet_number)
                # val = len(fit_pallets(matrix, swap(copy.deepcopy(poligons), i, j), eps))
                val = len(pal)
                if val < objVal:
                    stop = False
                    objVal = val
                    betterNeighboor = (i,j)
                    fit_pallets(matrix, swap(poligonsBest, i, j), eps)

                    # print(betterNeighboor)

                if betterNeighboor[1] != 0:
                    swap( poligonsBest, betterNeighboor[0], betterNeighboor[1])
        # print([item.pallet_number for item in poligonsBest])


    # print([item.pallet_number for item in items])
    # for i in range(len(pal)):
    #     print_matrix(pal[i])
    #     print('--------------')
    # understand_pallets(poligonsBest)

    # draw_all_pallets(understand_pallets(poligonsBest))

    return val, poligons

In [96]:
def crossover(parent1, parent2):
    child = parent1[:int(len(parent1)/2)]
    for i in range(len(parent1)):
        if parent2[i] not in child:
            child.append(parent2[i])
    # print([parent1[i].id for i in range(len(parent1))])
    # print([parent2[i].id for i in range(len(parent1))])
    # print([child[i].id for i in range(len(parent1))])
    return child

In [ ]:
record = len(items)
record_item = 0

population = []

k = 10 # размер изначальной популяции

# создаём начальную случайную популяцию и находим её рекорд
for i in range(k):
    population.append(sorted(items, key=lambda A: random.random()))
    result = fit_pallets(matrix, population[i], eps)
    if record > len(result):
        record = len(result)
        record_item = population[i]

# удаляем из списка рекорд
population.remove(record_item)

In [98]:
champions = population
start_time = time.time()
work_time = 0
while work_time < 10: # ограничение по времени работы 10 секунд
    for parent in population:
        crossover(record_item, parent)


    work_time = time.time() - start_time

IndentationError: expected an indented block (Temp/ipykernel_11756/434304262.py, line 8)